In [8]:
import arcpy, os
from arcpy import env
from arcpy.sa import *

In [2]:
arcpy.env.overwriteOutput = True

In [6]:
#arcpy.CheckExtension("Spatial Analyst")
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

In [5]:
env.workspace = r'T:\MPO\RTP\FY20 2045 Update\Data and Resources\Data\ForAppendixF\NaturalHazards\OSHD-1_GIS-bundle\OSHD Release 1_0.gdb'

In [10]:
mask = r"X:\Data\Transportation\MPO_Boundary.shp"

In [9]:
# extract by mask
outExtractByMask = ExtractByMask("Dry_Landslide_Susceptibility_Map", mask)
outExtractByMask.save("Dry_Landslide_Susceptibility_CLMPO")

In [11]:
outExtractByMask = ExtractByMask("Wet_Landslide_Susceptibility_Map", mask)
outExtractByMask.save("Wet_Landslide_Susceptibility_CLMPO")

In [12]:
outExtractByMask = ExtractByMask("Liquefaction_Susceptibility_Map", mask)
outExtractByMask.save("Liquefaction_Susceptibility_CLMPO")

In [13]:
outExtractByMask = ExtractByMask("Probability_of_Damaging_Shaking_Map", mask)
outExtractByMask.save("Probability_of_Damaging_Shaking_CLMPO")

In [15]:
# reclassify
outReclass1 = Reclassify("Dry_Landslide_Susceptibility_CLMPO", "Value", 
                         RemapRange([[0,4,0],[5,9,1]]), "NODATA")
outReclass1.save("Dry_Landslide_CLMPO_Reclass")

In [16]:
outReclass2 = Reclassify("Wet_Landslide_Susceptibility_CLMPO", "Value", 
                         RemapRange([[0,5,0],[6,10,1]]), "NODATA")
outReclass2.save("Wet_Landslide_CLMPO_Reclass")

In [17]:
outReclass3 = Reclassify("Liquefaction_Susceptibility_CLMPO", "Value", 
                         RemapRange([[0,2,0],[3,5,1]]), "NODATA")
outReclass3.save("Liquefaction_CLMPO_Reclass")

In [18]:
outReclass4 = Reclassify("Probability_of_Damaging_Shaking_CLMPO", "Value", 
                         RemapRange([[0,3,0],[4,5,1]]), "NODATA")
outReclass4.save("Damaging_Shaking_CLMPO_Reclass")

In [20]:
# raster to polygon
arcpy.RasterToPolygon_conversion("Dry_Landslide_CLMPO_Reclass", "Dry_Landslide_CLMPO_Reclass_Polygon", 
                                 "NO_SIMPLIFY", "Value")

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Data\\ForAppendixF\\NaturalHazards\\OSHD-1_GIS-bundle\\OSHD Release 1_0.gdb\\Dry_Landslide_CLMPO_Reclass_Polygon'>

In [21]:
arcpy.RasterToPolygon_conversion("Wet_Landslide_CLMPO_Reclass", "Wet_Landslide_CLMPO_Reclass_Polygon", 
                                 "NO_SIMPLIFY", "Value")

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Data\\ForAppendixF\\NaturalHazards\\OSHD-1_GIS-bundle\\OSHD Release 1_0.gdb\\Wet_Landslide_CLMPO_Reclass_Polygon'>

In [22]:
arcpy.RasterToPolygon_conversion("Liquefaction_CLMPO_Reclass", "Liquefaction_CLMPO_Reclass_Polygon", 
                                 "NO_SIMPLIFY", "Value")

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Data\\ForAppendixF\\NaturalHazards\\OSHD-1_GIS-bundle\\OSHD Release 1_0.gdb\\Liquefaction_CLMPO_Reclass_Polygon'>

In [23]:
arcpy.RasterToPolygon_conversion("Damaging_Shaking_CLMPO_Reclass", "Damaging_Shaking_CLMPO_Reclass_Polygon", 
                                 "NO_SIMPLIFY", "Value")

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Data\\ForAppendixF\\NaturalHazards\\OSHD-1_GIS-bundle\\OSHD Release 1_0.gdb\\Damaging_Shaking_CLMPO_Reclass_Polygon'>

In [24]:
shpfiles = ['Dry_Landslide', 'Wet_Landslide', 'Liquefaction', 'Damaging_Shaking']

In [33]:
# merge
for shpfile in shpfiles:
    fc = 'hazard_layer'
    arcpy.MakeFeatureLayer_management(shpfile+'_CLMPO_Reclass_Polygon', fc)
    arcpy.SelectLayerByAttribute_management(fc, 'NEW_SELECTION', '"gridcode"=1')
    #arcpy.CopyFeatures_management('hazard_layer', shpfile)
    field = 'source'
    arcpy.AddField_management(fc, field,  "TEXT", "", "", 50)
    with arcpy.da.UpdateCursor(fc, field) as cursor:
        for row in cursor:
            row[0] = shpfile
            cursor.updateRow(row)
    if shpfile == 'Dry_Landslide':
        arcpy.CopyFeatures_management(fc, 'earthquake_layer')
    else:
        arcpy.Append_management(fc, 'earthquake_layer', "TEST","","")

In [30]:
outpath = r'T:\MPO\RTP\FY20 2045 Update\Data and Resources\Data\ForAppendixF'

In [34]:
# dissolve
arcpy.Dissolve_management('earthquake_layer', os.path.join(outpath, 'NaturalHazards', 'EarthquakeLayer.shp'), 'source')

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Data\\ForAppendixF\\NaturalHazards\\EarthquakeLayer.shp'>